# CREATE DATASET WITH TOP SONGS LISTS FROM PREVIOUS LABS

### IMPORT LIBRARIES

In [229]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import config

### SPOTIPY CONNECTION

In [230]:
client_credentials_manager = SpotifyClientCredentials(client_id=config.client_id, client_secret=config.client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=120)

### LOAD DATA

In [231]:
billboard = pd.read_csv('df_billboard100.csv')
npr = pd.read_csv('df_npr.csv')
global50 = pd.read_csv('df_top50global.csv')

In [232]:
billboard.head()

,current_rank,song_title,artist_name,last_week_rank
0,1,Last Night,Morgan Wallen,1
1,2,Fast Car,Luke Combs,3
2,3,Calm Down,Rema & Selena Gomez,4
3,4,Flowers,Miley Cyrus,2
4,5,All My Life,Lil Durk Featuring J. Cole,5


In [233]:
npr.head()

,artist,song_name
0,Little Simz,"""Gorilla"""
1,Ian William Craig,"""Attention For It Radiates"""
2,Viking Ding Dong x Ravi B,"""Leave It Alone (Remix)"""
3,Adeem the Artist,"""Middle of a Heart"""
4,"Zahsosaa, D STURDY & DJ Crazy","""Shakedhat"""


In [234]:
npr.columns = ["artist_name", "song_title"]

In [235]:
global50.head()

,type,id,name,artist_name,shareUrl,addedAt,durationMs,durationText,discNumber,trackNumber,album_name,album_id,trackCount
0,track,3qQbCzHBycnDpGskqOWY0E,Ella Baila Sola,Eslabon Armado,https://open.spotify.com/track/3qQbCzHBycnDpGs...,2023-06-30T10:18:05Z,165671,02:45,1,6,DESVELADO,5aDEezKnOqyQo0qvTFhpkM,16
1,track,7ro0hRteUMfnOioTFI5TG1,WHERE SHE GOES,Bad Bunny,https://open.spotify.com/track/7ro0hRteUMfnOio...,2023-06-30T10:18:05Z,231704,03:51,1,1,WHERE SHE GOES,5gCcb5fsSb6w5K8SyJrgtB,1
2,track,2UW7JaomAMuX9pZrjVpHAU,La Bebe - Remix,Yng Lvcas,https://open.spotify.com/track/2UW7JaomAMuX9pZ...,2023-06-30T10:18:05Z,234352,03:54,1,1,La Bebe (Remix),6aBVGuOUEuX18rHxyDWbti,2
3,track,6pD0ufEQq0xdHSsRbg9LBK,un x100to,Grupo Frontera,https://open.spotify.com/track/6pD0ufEQq0xdHSs...,2023-06-30T10:18:05Z,194563,03:14,1,1,un x100to,7aGzSSUD8S6IhPCsZSiuMT,1
4,track,1BxfuPKGuaTgP7aM0Bbdwr,Cruel Summer,Taylor Swift,https://open.spotify.com/track/1BxfuPKGuaTgP7a...,2023-06-30T10:18:05Z,178426,02:58,1,2,Lover,1NAmidJlEaVgA3MpcPFYGq,18


In [236]:
# get only song title and artist name from the billboard and concat with npr dataframe. 
# The global50 dataset already have the track_ids so we will integrate it later once we need to extract the audio features.
billboard1 = billboard[["song_title", "artist_name"]]

In [237]:
global50_1 = global50[['id', 'name', "artist_name"]]
global50_1.columns = ['track_id', 'song_title', "artist_name"]

In [238]:
top_songs = pd.concat([billboard1, npr])
top_songs

,song_title,artist_name
0,Last Night,Morgan Wallen
1,Fast Car,Luke Combs
2,Calm Down,Rema & Selena Gomez
3,Flowers,Miley Cyrus
4,All My Life,Lil Durk Featuring J. Cole
...,...,...
95,"""Hype Boy""",NewJeans
96,"""Feminina""",Joyce
97,"""Rush""",Ayra Starr
98,"""Waterfall""",Disclosure feat. RAYE


In [239]:
# clean song_tile column by removing quotation marks
top_songs.song_title = top_songs.song_title.replace('"', "", regex=True)

In [240]:
top_songs

,song_title,artist_name
0,Last Night,Morgan Wallen
1,Fast Car,Luke Combs
2,Calm Down,Rema & Selena Gomez
3,Flowers,Miley Cyrus
4,All My Life,Lil Durk Featuring J. Cole
...,...,...
95,Hype Boy,NewJeans
96,Feminina,Joyce
97,Rush,Ayra Starr
98,Waterfall,Disclosure feat. RAYE


In [241]:
top_songs.duplicated().sum()

80

In [242]:
top_songs.drop_duplicates(inplace=True)

In [243]:
top_songs.shape

(120, 2)

### FUNCTIONS

In [244]:
def get_track_ids(songlist, artistlist):
    track_ids = []
    try:
        for s,a in zip(songlist, artistlist):
            search = sp.search(q= str(s) + " " + str(a), type="track", limit=1)
            track_ids.append(search["tracks"]['items'][0]["id"] if search["tracks"]['items'][0]["id"] else 'error')
    except (IndexError, TypeError):
        pass
    return track_ids

In [245]:
def audio_features(track_ids_list):
    audio_features = pd.DataFrame()
    for i, o in zip(range(0, (len(track_ids_list)+100), 100), range(100, (len(track_ids_list)+100), 100)):
        try:
            features = sp.audio_features(track_ids_list[i:o])
            features_df = pd.DataFrame(features)
            audio_features = pd.concat([audio_features, features_df])
        except AttributeError:
            pass
    return audio_features

#### GETTING TRACK IDS

In [246]:
song_list1 = get_track_ids(top_songs.song_title, top_songs.artist_name)

In [247]:
len(song_list1)

120

In [248]:
top_songs.insert(0, 'track_id', song_list1)

In [249]:
top_songs

,track_id,song_title,artist_name
0,59uQI0PADDKeE6UZDTJEe8,Last Night,Morgan Wallen
1,1Lo0QY9cvc8sUB2vnIOxDT,Fast Car,Luke Combs
2,1s7oOCT8vauUh01PbJD6ps,Calm Down,Rema & Selena Gomez
3,0yLdNVWF3Srea0uzk55zFn,Flowers,Miley Cyrus
4,6HgWWaMu31KdOpEG5l28BG,All My Life,Lil Durk Featuring J. Cole
...,...,...,...
15,0a4MMyCrzT0En247IhqZbD,Hype Boy,NewJeans
16,1BUWbJwPCBcTLiQBE5gBdI,Feminina,Joyce
17,1rrqJ9QkOBYJlsZgqqwxgB,Rush,Ayra Starr
18,3X2Aw6bQ7TfMMKsTzjTlDg,Waterfall,Disclosure feat. RAYE


In [250]:
top_songs = pd.concat([top_songs, global50_1])
top_songs

,track_id,song_title,artist_name
0,59uQI0PADDKeE6UZDTJEe8,Last Night,Morgan Wallen
1,1Lo0QY9cvc8sUB2vnIOxDT,Fast Car,Luke Combs
2,1s7oOCT8vauUh01PbJD6ps,Calm Down,Rema & Selena Gomez
3,0yLdNVWF3Srea0uzk55zFn,Flowers,Miley Cyrus
4,6HgWWaMu31KdOpEG5l28BG,All My Life,Lil Durk Featuring J. Cole
...,...,...,...
45,26b3oVLrRUaaybJulow9kz,People,Libianca
46,3JvKfv6T31zO0ini8iNItO,Another Love,Tom Odell
47,4rXLjWdF2ZZpXCVTfWcshS,fukumean,Gunna
48,4FAKtPVycI4DxoOHC01YqD,Yandel 150,Yandel


In [251]:
# rechecking for duplicates since we joined another dataframe
top_songs.duplicated().sum() # drop 6 duplicated
top_songs.drop_duplicates(inplace=True)

In [252]:
top_songs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 164 entries, 0 to 49
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   track_id     164 non-null    object
 1   song_title   164 non-null    object
 2   artist_name  164 non-null    object
dtypes: object(3)
memory usage: 5.1+ KB


#### GETTING AUDIO FEATURES

In [253]:
audio_feat = audio_features(top_songs.track_id)

In [254]:
audio_feat

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.517,0.675,6,-5.382,1,0.0357,0.4590,0.000000,0.1510,0.518,203.853,audio_features,59uQI0PADDKeE6UZDTJEe8,spotify:track:59uQI0PADDKeE6UZDTJEe8,https://api.spotify.com/v1/tracks/59uQI0PADDKe...,https://api.spotify.com/v1/audio-analysis/59uQ...,163855,4
1,0.712,0.603,8,-5.520,1,0.0262,0.1860,0.000000,0.1150,0.670,97.994,audio_features,1Lo0QY9cvc8sUB2vnIOxDT,spotify:track:1Lo0QY9cvc8sUB2vnIOxDT,https://api.spotify.com/v1/tracks/1Lo0QY9cvc8s...,https://api.spotify.com/v1/audio-analysis/1Lo0...,265493,4
2,0.799,0.802,11,-5.196,1,0.0371,0.4290,0.001280,0.1710,0.811,107.008,audio_features,1s7oOCT8vauUh01PbJD6ps,spotify:track:1s7oOCT8vauUh01PbJD6ps,https://api.spotify.com/v1/tracks/1s7oOCT8vauU...,https://api.spotify.com/v1/audio-analysis/1s7o...,239318,4
3,0.707,0.681,0,-4.325,1,0.0668,0.0632,0.000005,0.0322,0.646,117.999,audio_features,0yLdNVWF3Srea0uzk55zFn,spotify:track:0yLdNVWF3Srea0uzk55zFn,https://api.spotify.com/v1/tracks/0yLdNVWF3Sre...,https://api.spotify.com/v1/audio-analysis/0yLd...,200455,4
4,0.829,0.436,3,-8.205,1,0.3270,0.1500,0.000000,0.0954,0.693,143.031,audio_features,6HgWWaMu31KdOpEG5l28BG,spotify:track:6HgWWaMu31KdOpEG5l28BG,https://api.spotify.com/v1/tracks/6HgWWaMu31Kd...,https://api.spotify.com/v1/audio-analysis/6HgW...,223204,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,0.617,0.589,7,-5.978,1,0.0345,0.0268,0.000000,0.3780,0.566,92.991,audio_features,5IAESfJjmOYu7cHyX557kz,spotify:track:5IAESfJjmOYu7cHyX557kz,https://api.spotify.com/v1/tracks/5IAESfJjmOYu...,https://api.spotify.com/v1/audio-analysis/5IAE...,229953,4
60,0.445,0.537,4,-8.532,0,0.0400,0.6950,0.000017,0.0944,0.131,122.769,audio_features,3JvKfv6T31zO0ini8iNItO,spotify:track:3JvKfv6T31zO0ini8iNItO,https://api.spotify.com/v1/tracks/3JvKfv6T31zO...,https://api.spotify.com/v1/audio-analysis/3JvK...,244360,4
61,0.847,0.622,1,-6.747,0,0.0903,0.1190,0.000000,0.2850,0.220,130.001,audio_features,4rXLjWdF2ZZpXCVTfWcshS,spotify:track:4rXLjWdF2ZZpXCVTfWcshS,https://api.spotify.com/v1/tracks/4rXLjWdF2ZZp...,https://api.spotify.com/v1/audio-analysis/4rXL...,125040,4
62,0.783,0.729,6,-3.549,0,0.0691,0.0492,0.000272,0.1000,0.580,167.968,audio_features,4FAKtPVycI4DxoOHC01YqD,spotify:track:4FAKtPVycI4DxoOHC01YqD,https://api.spotify.com/v1/tracks/4FAKtPVycI4D...,https://api.spotify.com/v1/audio-analysis/4FAK...,216148,4


In [255]:
audio_feat.columns

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature'],
      dtype='object')

In [256]:
cols_to_drop = ['type', 'uri', 'track_href', 'analysis_url', 'time_signature']

In [257]:
audio_features = audio_feat.drop(columns=cols_to_drop)

In [258]:
top_songs

,track_id,song_title,artist_name
0,59uQI0PADDKeE6UZDTJEe8,Last Night,Morgan Wallen
1,1Lo0QY9cvc8sUB2vnIOxDT,Fast Car,Luke Combs
2,1s7oOCT8vauUh01PbJD6ps,Calm Down,Rema & Selena Gomez
3,0yLdNVWF3Srea0uzk55zFn,Flowers,Miley Cyrus
4,6HgWWaMu31KdOpEG5l28BG,All My Life,Lil Durk Featuring J. Cole
...,...,...,...
44,5IAESfJjmOYu7cHyX557kz,Take Two,BTS
46,3JvKfv6T31zO0ini8iNItO,Another Love,Tom Odell
47,4rXLjWdF2ZZpXCVTfWcshS,fukumean,Gunna
48,4FAKtPVycI4DxoOHC01YqD,Yandel 150,Yandel


In [260]:
top_song_list = top_songs.merge(audio_features,  left_on="track_id", right_on="id",)

In [261]:
#duplicates?
top_song_list.duplicated().sum()

26

In [262]:
top_tracks = top_song_list.drop_duplicates()

In [263]:
top_tracks

,track_id,song_title,artist_name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms
0,59uQI0PADDKeE6UZDTJEe8,Last Night,Morgan Wallen,0.517,0.675,6,-5.382,1,0.0357,0.45900,0.000000,0.1510,0.518,203.853,59uQI0PADDKeE6UZDTJEe8,163855
1,1Lo0QY9cvc8sUB2vnIOxDT,Fast Car,Luke Combs,0.712,0.603,8,-5.520,1,0.0262,0.18600,0.000000,0.1150,0.670,97.994,1Lo0QY9cvc8sUB2vnIOxDT,265493
2,1s7oOCT8vauUh01PbJD6ps,Calm Down,Rema & Selena Gomez,0.799,0.802,11,-5.196,1,0.0371,0.42900,0.001280,0.1710,0.811,107.008,1s7oOCT8vauUh01PbJD6ps,239318
4,1s7oOCT8vauUh01PbJD6ps,Calm Down (with Selena Gomez),Rema,0.799,0.802,11,-5.196,1,0.0371,0.42900,0.001280,0.1710,0.811,107.008,1s7oOCT8vauUh01PbJD6ps,239318
6,0yLdNVWF3Srea0uzk55zFn,Flowers,Miley Cyrus,0.707,0.681,0,-4.325,1,0.0668,0.06320,0.000005,0.0322,0.646,117.999,0yLdNVWF3Srea0uzk55zFn,200455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,4lIxdJw6W3Fg4vUIYCB0S5,Style,Taylor Swift,0.598,0.786,2,-5.572,1,0.0383,0.00253,0.001600,0.1170,0.456,95.019,4lIxdJw6W3Fg4vUIYCB0S5,231000
186,5IAESfJjmOYu7cHyX557kz,Take Two,BTS,0.617,0.589,7,-5.978,1,0.0345,0.02680,0.000000,0.3780,0.566,92.991,5IAESfJjmOYu7cHyX557kz,229953
187,3JvKfv6T31zO0ini8iNItO,Another Love,Tom Odell,0.445,0.537,4,-8.532,0,0.0400,0.69500,0.000017,0.0944,0.131,122.769,3JvKfv6T31zO0ini8iNItO,244360
188,4FAKtPVycI4DxoOHC01YqD,Yandel 150,Yandel,0.783,0.729,6,-3.549,0,0.0691,0.04920,0.000272,0.1000,0.580,167.968,4FAKtPVycI4DxoOHC01YqD,216148


In [264]:
# save to csv
top_tracks.to_csv('top_tracks.csv', index=False)